In [36]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [57]:
# Importing Libraries
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from statsmodels import api as sm
import numpy as np
from tqdm import tqdm

In [38]:
# Importing the dataset
df=pd.read_csv('/content/drive/MyDrive/DA2023/train.csv')

In [39]:
# Filling null values in 'Id' column
for i in df.columns[2:]:
  df[i] = df[i].fillna(df["Id"])

In [40]:
# Making 'target' dataframe
target = df[df["Time"].isna()]["Sensor_1"].reset_index(drop=True)

In [41]:
# Dropping irrelevant columns
df.pop('Id')
df.pop('Time')

0          1.0
1          2.0
2          3.0
3          4.0
4          5.0
          ... 
549995     7.0
549996     8.0
549997     9.0
549998    10.0
549999     NaN
Name: Time, Length: 550000, dtype: float64

In [42]:
# Checking columns left in training data
df.columns

Index(['Sensor_1', 'Sensor_2', 'Sensor_3', 'Sensor_4', 'Sensor_5', 'Sensor_6',
       'Sensor_7', 'Sensor_8', 'Sensor_9', 'Sensor_10', 'Sensor_11',
       'Sensor_12', 'Sensor_13', 'Sensor_14', 'Sensor_15', 'Sensor_16',
       'Sensor_17', 'Sensor_18', 'Sensor_19', 'Sensor_20', 'Sensor_21',
       'Sensor_22', 'Sensor_23', 'Sensor_24', 'Sensor_25'],
      dtype='object')

In [43]:
# Preparing training set
sequences_train = list()
for i in range(0,30000):
    df_temp = df.iloc[(i*11):((i+1)*11)-1]
    values = df_temp.values
    values = values.transpose()
    sequences_train.append(values)

target_train=target[0:30000]

In [44]:
# Preparing target dataframe of training set
sequences_test = list()
for i in range(30000,40000):
    df_temp = df.iloc[(i*11):((i+1)*11)-1]
    values = df_temp.values
    values = values.transpose()
    sequences_test.append(values)

target_test=target[30000:40000]

In [45]:
# Size of dataset
len(df)

550000

In [46]:
# Preparing validation set
sequences_val = list()
for i in range(40000,49500):
    df_temp = df.iloc[(i*11):((i+1)*11)-1]
    values = df_temp.values
    values = values.transpose()
    sequences_val.append(values)

target_val=target[40000:49500]

In [47]:
# Converting them into array for fitting into LSTM model
sequence_train=np.array(sequences_train)
sequence_test=np.array(sequences_test)
sequence_val=np.array(sequences_val)

In [48]:
# Importing required DL libraries
from keras.preprocessing import sequence
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import Adam
from keras.models import load_model
from keras.callbacks import ModelCheckpoint
from keras.utils import to_categorical
from keras.layers import TimeDistributed
from keras.layers import Bidirectional

In [49]:
# Converting targets of all sets to categorical
targets_train=to_categorical(target_train, num_classes=4)
targets_test=to_categorical(target_test, num_classes=4)
targets_val=to_categorical(target_val, num_classes=4)

In [50]:
# Preparing model
model = Sequential()
model.add(Bidirectional(LSTM(64, return_sequences=True, input_shape=(25, 10))))
#model.add(BatchNormalization())
model.add(Bidirectional(LSTM(64, return_sequences=True))) # returns a sequence of vectors of dimension 32
#model.add(BatchNormalization())
model.add(Bidirectional(LSTM(32, return_sequences=True)))  # returns a sequence of vectors of dimension 32
#model.add(BatchNormalization())
model.add(Bidirectional(LSTM(32)))
#model.add(BatchNormalization())
model.add(Dense(4, activation='softmax'))

In [51]:
# Fitting model
adam = Adam(lr=0.001)
chk = ModelCheckpoint('best_model.pkl', monitor='val_acc', save_best_only=True, mode='max', verbose=1)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
model.fit(sequence_train, targets_train, epochs=100, batch_size=128, callbacks=[chk], validation_data=(sequence_val,targets_val))

Epoch 1/100
235/235 [==============================] - ETA: 0s - loss: 1.3746 - accuracy: 0.2941

235/235 [==============================] - 98s 330ms/step - loss: 1.3746 - accuracy: 0.2941 - val_loss: 1.3645 - val_accuracy: 0.3028
Epoch 2/100
235/235 [==============================] - ETA: 0s - loss: 1.3351 - accuracy: 0.3390

235/235 [==============================] - 68s 289ms/step - loss: 1.3351 - accuracy: 0.3390 - val_loss: 1.3027 - val_accuracy: 0.3674
Epoch 3/100
235/235 [==============================] - ETA: 0s - loss: 1.2971 - accuracy: 0.3702

235/235 [==============================] - 66s 279ms/step - loss: 1.2971 - accuracy: 0.3702 - val_loss: 1.2594 - val_accuracy: 0.3973
Epoch 4/100
235/235 [==============================] - ETA: 0s - loss: 1.1482 - accuracy: 0.4582

235/235 [==============================] - 71s 303ms/step - loss: 1.1482 - accuracy: 0.4582 - val_loss: 1.0773 - val_accuracy: 0.5023
Epoch 5/100
235/235 [==============================] - ETA: 0s - loss: 0.9275 - accuracy: 0.5748

...

235/235 [==============================] - 67s 285ms/step - loss: 0.0712 - accuracy: 0.9733 - val_loss: 0.2615 - val_accuracy: 0.9079
Epoch 97/100
235/235 [==============================] - ETA: 0s - loss: 0.0687 - accuracy: 0.9740

235/235 [==============================] - 76s 326ms/step - loss: 0.0687 - accuracy: 0.9740 - val_loss: 0.1636 - val_accuracy: 0.9363
Epoch 98/100
235/235 [==============================] - ETA: 0s - loss: 0.0611 - accuracy: 0.9761

235/235 [==============================] - 71s 304ms/step - loss: 0.0611 - accuracy: 0.9761 - val_loss: 0.2253 - val_accuracy: 0.9211
Epoch 99/100
235/235 [==============================] - ETA: 0s - loss: 0.0651 - accuracy: 0.9744

235/235 [==============================] - 73s 313ms/step - loss: 0.0651 - accuracy: 0.9744 - val_loss: 0.2516 - val_accuracy: 0.9134
Epoch 100/100
235/235 [==============================] - ETA: 0s - loss: 0.0606 - accuracy: 0.9766

235/235 [==============================] - 70s 298ms/step - loss: 0.0606 - accuracy: 0.9766 - val_loss: 0.2481 - val_accuracy: 0.9153


In [52]:
# Perform prediction
from sklearn.metrics import accuracy_score
test_preds = model.predict(sequence_test)

313/313 [==============================] - 10s 22ms/step


In [58]:
# Print predictions
print(test_preds)

[[2.4423380e-06 1.5146946e-11 1.8166912e-07 9.9999732e-01]
 [5.0586678e-08 1.1777139e-06 9.7135240e-01 2.8646357e-02]
 [3.0340531e-01 6.9659448e-01 1.6936561e-07 4.7643067e-09]
 ...
 [9.9999803e-01 9.2646405e-07 2.8643925e-12 9.4428378e-07]
 [1.6292623e-04 9.9983668e-01 3.5882795e-07 3.5367671e-11]
 [1.2521541e-08 2.8557284e-07 9.6198845e-01 3.8011149e-02]]


The obtained predictions needs to be converted into one among {1,2,3,4}

In [54]:
# Converting predictions
preds=[]
for i in range(len(target_test)):
  if test_preds[i][0]>=test_preds[i][1]:
    if test_preds[i][0]>=test_preds[i][2]:
      if test_preds[i][0]>=test_preds[i][3]:
        preds.append(0)
  if test_preds[i][1]>=test_preds[i][0]:
    if test_preds[i][1]>=test_preds[i][2]:
      if test_preds[i][1]>=test_preds[i][3]:
        preds.append(1)
  if test_preds[i][2]>=test_preds[i][1]:
    if test_preds[i][2]>=test_preds[i][0]:
      if test_preds[i][2]>=test_preds[i][3]:
        preds.append(2)
  if test_preds[i][3]>=test_preds[i][1]:
    if test_preds[i][3]>=test_preds[i][2]:
      if test_preds[i][3]>=test_preds[i][0]:
        preds.append(3)

In [55]:
# Converted predictions
target_test

30000    3.0
30001    2.0
30002    1.0
30003    2.0
30004    1.0
        ... 
39995    3.0
39996    2.0
39997    0.0
39998    1.0
39999    2.0
Name: Sensor_1, Length: 10000, dtype: float64

30000    3.0
30001    2.0
30002    1.0
30003    2.0
30004    1.0
        ... 
39995    3.0
39996    2.0
39997    0.0
39998    1.0
39999    2.0
Name: Sensor_1, Length: 10000, dtype: float64

In [56]:
# Accuracy score
accuracy_score(target_test,preds)

0.9108